In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
data = pd.read_csv("../Data/cleaned_metadata.csv")
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head()

,offset,sex,age,RT_PCR_positive,survival,intubated,intubation_present,went_icu,in_icu,view,...,Pneumocystis,Pneumonia,SARS,Staphylococcus,Streptococcus,Tuberculosis,Unknown,Varicella,Viral,todo
0,0.0,M,65.0,Y,Y,N,N,N,N,PA,...,0,1,0,0,0,0,0,0,1,0
1,3.0,M,65.0,Y,Y,N,N,N,N,PA,...,0,1,0,0,0,0,0,0,1,0
2,5.0,M,65.0,Y,Y,N,N,N,N,PA,...,0,1,0,0,0,0,0,0,1,0
3,6.0,M,65.0,Y,Y,N,N,N,N,PA,...,0,1,0,0,0,0,0,0,1,0
4,0.0,F,52.0,Y,NaN,N,N,N,N,PA,...,0,1,0,0,0,0,0,0,1,0


In [3]:
data.describe()

,offset,age,Aspergillosis,Aspiration,Bacterial,COVID-19,Chlamydophila,E.Coli,Fungal,H1N1,...,Pneumocystis,Pneumonia,SARS,Staphylococcus,Streptococcus,Tuberculosis,Unknown,Varicella,Viral,todo
count,687.000000,692.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,...,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000
mean,9.147016,53.656069,0.002153,0.001076,0.078579,0.606028,0.003229,0.004306,0.034446,0.002153,...,0.032293,0.866523,0.017223,0.001076,0.023681,0.019376,0.001076,0.006459,0.651238,0.089343
std,31.885641,16.992385,0.046374,0.032809,0.269226,0.488892,0.056765,0.065512,0.182469,0.046374,...,0.176872,0.340273,0.130171,0.032809,0.152136,0.137916,0.032809,0.080148,0.476835,0.285392
min,-360.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,55.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,10.000000,68.250000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,365.000000,94.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   offset              687 non-null    float64
 1   sex                 849 non-null    object 
 2   age                 692 non-null    float64
 3   RT_PCR_positive     572 non-null    object 
 4   survival            361 non-null    object 
 5   intubated           247 non-null    object 
 6   intubation_present  249 non-null    object 
 7   went_icu            396 non-null    object 
 8   in_icu              334 non-null    object 
 9   view                929 non-null    object 
 10  modality            929 non-null    object 
 11  filename            929 non-null    object 
 12  Aspergillosis       929 non-null    int64  
 13  Aspiration          929 non-null    int64  
 14  Bacterial           929 non-null    int64  
 15  COVID-19            929 non-null    int64  
 16  Chlamydo

In [11]:
data_copy = data.copy()

In [12]:
for column in data_copy.columns:
    if data_copy[column].dtype == 'O' and len(data_copy[column].unique()) < 4:
        print("{} unique values: {}".format(column, data_copy[column].unique()))
        data_copy[column].replace({'N':0, 'Y':1, 'M':0, 'F':1, 'X-ray':0, 'CT':1, 'Unclear':0}, inplace=True)
        print("After conversion unique values: {}".format(data_copy[column].unique()))
    elif data_copy[column].dtype == 'O' and len(data_copy[column].unique()) >= 4 and len(data_copy[column].unique()) <= 10:
        print("{} unique values: {}".format(column, data_copy[column].unique()))
        data_copy = pd.get_dummies(data_copy, columns=['view'], prefix=['view'])
        print(data_copy.columns)
        # print(data_copy.shape)
        # data_copy.drop(['view'], inplace=True, axis=1)
        # data_copy = pd.concat([data_copy, view], axis=1)
        # print(data_copy.shape)

sex unique values: ['M' 'F' nan]
After conversion unique values: [ 0.  1. nan]
RT_PCR_positive unique values: ['Y' nan 'Unclear']
After conversion unique values: [ 1. nan  0.]
survival unique values: ['Y' nan 'N']
After conversion unique values: [ 1. nan  0.]
intubated unique values: ['N' 'Y' nan]
After conversion unique values: [ 0.  1. nan]
intubation_present unique values: ['N' 'Y' nan]
After conversion unique values: [ 0.  1. nan]
went_icu unique values: ['N' 'Y' nan]
After conversion unique values: [ 0.  1. nan]
in_icu unique values: ['N' 'Y' nan]
After conversion unique values: [ 0.  1. nan]
view unique values: ['PA' 'AP' 'L' 'Axial' 'AP Supine' 'Coronal' 'AP Erect']
Index(['offset', 'sex', 'age', 'RT_PCR_positive', 'survival', 'intubated',
       'intubation_present', 'went_icu', 'in_icu', 'modality', 'filename',
       'Aspergillosis', 'Aspiration', 'Bacterial', 'COVID-19', 'Chlamydophila',
       'E.Coli', 'Fungal', 'H1N1', 'Herpes ', 'Influenza', 'Klebsiella',
       'Legione

In [13]:
data_copy = data_copy[['offset', 'sex', 'age', 'RT_PCR_positive', 'survival', 'intubated',
       'intubation_present', 'went_icu', 'in_icu', 'view_AP', 'view_AP Erect', 'view_AP Supine', 'view_Axial',
       'view_Coronal', 'view_L', 'view_PA', 'modality', 'filename',
       'Aspergillosis', 'Aspiration', 'Bacterial', 'COVID-19', 'Chlamydophila',
       'E.Coli', 'Fungal', 'H1N1', 'Herpes ', 'Influenza', 'Klebsiella',
       'Legionella', 'Lipoid', 'MERS-CoV', 'MRSA', 'Mycoplasma', 'No Finding',
       'Nocardia', 'Pneumocystis', 'Pneumonia', 'SARS', 'Staphylococcus',
       'Streptococcus', 'Tuberculosis', 'Unknown', 'Varicella', 'Viral',
       'todo']]

data_copy.columns

Index(['offset', 'sex', 'age', 'RT_PCR_positive', 'survival', 'intubated',
       'intubation_present', 'went_icu', 'in_icu', 'view_AP', 'view_AP Erect',
       'view_AP Supine', 'view_Axial', 'view_Coronal', 'view_L', 'view_PA',
       'modality', 'filename', 'Aspergillosis', 'Aspiration', 'Bacterial',
       'COVID-19', 'Chlamydophila', 'E.Coli', 'Fungal', 'H1N1', 'Herpes ',
       'Influenza', 'Klebsiella', 'Legionella', 'Lipoid', 'MERS-CoV', 'MRSA',
       'Mycoplasma', 'No Finding', 'Nocardia', 'Pneumocystis', 'Pneumonia',
       'SARS', 'Staphylococcus', 'Streptococcus', 'Tuberculosis', 'Unknown',
       'Varicella', 'Viral', 'todo'],
      dtype='object')

In [16]:
scale = StandardScaler()
data_copy['offset_standardized'] = scale.fit_transform(np.array(data_copy['offset']).reshape(-1,1))

In [18]:
data_copy['offset_standardized'].describe(), data_copy['offset'].describe()

(count    6.870000e+02
 mean    -5.171345e-17
 std      1.000729e+00
 min     -1.158565e+01
 25%     -2.556935e-01
 50%     -1.301538e-01
 75%      2.677084e-02
 max      1.116842e+01
 Name: offset_standardized, dtype: float64,
 count    687.000000
 mean       9.147016
 std       31.885641
 min     -360.000000
 25%        1.000000
 50%        5.000000
 75%       10.000000
 max      365.000000
 Name: offset, dtype: float64)

In [19]:
len(data_copy['age'].unique())

70